In [82]:
import spacy
import re

# Wczytaj model SpaCy dla języka polskiego
nlp = spacy.load("pl_core_news_sm")

# Funkcja do podziału na sylaby
def podziel_na_sylaby(slowo):
    # Wzorzec do znajdowania sylab: samogłoska + opcjonalne spółgłoski
    wzorzec = r"[aeiouyąę]{1,2}[^aeiouyąę]*"
    sylaby = re.findall(wzorzec, slowo, re.IGNORECASE)
    return sylaby

# Wczytaj plik z tekstem
with open("pan-tadeusz.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Przetwarzanie tekstu za pomocą SpaCy
doc = nlp(text)

# Iteracja po tokenach i podział na sylaby
print("Podział tokenów na sylaby:")
for token in doc[:20]:  # Przykład dla pierwszych 20 tokenów
    if token.is_alpha:  # Sprawdzamy, czy token jest słowem
        sylaby = podziel_na_sylaby(token.text)
        print(f"Słowo: {token.text}, Sylaby: {sylaby}")


Podział tokenów na sylaby:
Słowo: Litwo, Sylaby: ['itw', 'o']
Słowo: Ojczyzno, Sylaby: ['Ojcz', 'yzn', 'o']
Słowo: moja, Sylaby: ['oj', 'a']
Słowo: ty, Sylaby: ['y']
Słowo: jesteś, Sylaby: ['est', 'eś']
Słowo: jak, Sylaby: ['ak']
Słowo: zdrowie, Sylaby: ['ow', 'ie']
Słowo: Ile, Sylaby: ['Il', 'e']
Słowo: cię, Sylaby: ['ię']
Słowo: trzeba, Sylaby: ['eb', 'a']
Słowo: cenić, Sylaby: ['en', 'ić']
Słowo: ten, Sylaby: ['en']
Słowo: tylko, Sylaby: ['ylk', 'o']
Słowo: się, Sylaby: ['ię']
Słowo: dowie, Sylaby: ['ow', 'ie']


In [66]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt

# hyperparameters
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 64  # maximum context length for predictions
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.1

# dynamic block_size adjustment for GPU memory
#if device == 'cuda':
#device = "cuda"
gpu_memory = torch.cuda.get_device_properties(0).total_memory
block_size = min(128, max(16, gpu_memory // (n_embd * 1024)))

torch.manual_seed(166045)

# Load text data
with open('pan-tadeusz-bpe.txt', 'r', encoding='utf-8') as f:
    text = f.read()


# Inicjalizacja i trening tokenizera BPE
bpe_tokenizer = BPETokenizer(vocab_size=10000)  # Określ wielkość słownika
bpe_tokenizer.train(text)  # Trenuj tokenizer na korpusie tekstowym

vocab_size = len(bpe_tokenizer.vocab)


# Sprawdź, czy słownik został poprawnie wytrenowany
if bpe_tokenizer.vocab is None:
    raise ValueError("Tokenizer vocab is None. Make sure to train the tokenizer before using it.")

# Utwórz słownik tokenów na podstawie wytrenowanego tokenizera
tokens = list(bpe_tokenizer.vocab)  # Lista tokenów w słowniku BPE
stoi = {token: i for i, token in enumerate(tokens)}
itos = {i: token for token, i in stoi.items()}

# Zmień funkcje `encode` i `decode` na operujące na liczbach
encode = lambda s: [stoi[token] for token in bpe_tokenizer.encode(s)]

def decode(token_ids):
    tokens = [itos[token_id] for token_id in token_ids]  # Zamień ID na tokeny
    return bpe_tokenizer.decode(tokens)  # Użyj funkcji `decode` z BPE


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split, seq_sampling=False):
    data = train_data if split == 'train' else val_data
    if seq_sampling:
        start_idx = torch.randint(0, len(data) - block_size, (1,)).item()
        x = data[start_idx:start_idx + block_size].unsqueeze(0).repeat(batch_size, 1)
        y = data[start_idx + 1:start_idx + block_size + 1].unsqueeze(0).repeat(batch_size, 1)
    else:
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = torch.stack([data[i:i + block_size] for i in ix])
        y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = x + self.dropout(self.sa(self.ln1(x)))
        x = x + self.dropout(self.ffwd(self.ln2(x)))
        return x

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = GPTModel().to(device)
print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

train_losses, val_losses = [], []

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        train_losses.append(losses['train'])
        val_losses.append(losses['val'])
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.legend()
plt.xlabel('Evaluation Interval')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.show()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=2000)[0].tolist())

with open('wynik_oczyszczony.txt', 'w', encoding='utf-8') as f:
    f.write(generated_text)


KeyboardInterrupt: 